In [41]:
import numpy as np
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go

# Define the transformation matrix M
M = np.array([[0.6, 0.3], [-0.2, 1.4]])

# Function to generate sequences
def generate_sequence(point, matrix, steps=40):
    sequence = [point]
    for _ in range(steps - 1):
        point = matrix @ point
        sequence.append(point)
    return np.array(sequence)

# Create Dash app
app = dash.Dash(__name__)

# Layout of the app
app.layout = html.Div([
    html.H1("Eigenspaces and Dynamical Systems"),
    dcc.Graph(
        id="2d-plot",
        config={"scrollZoom": True}  # Enables zooming and panning
    ),
    html.Div([
        html.Label("Adjust Coordinates of Point A:"),
        dcc.Slider(id="slider-ax", min=-60, max=60, step=0.1, value=28.82,
                   marks={i: str(i) for i in range(-60, 61, 20)}),
        dcc.Slider(id="slider-ay", min=-60, max=60, step=0.1, value=-0.86,
                   marks={i: str(i) for i in range(-60, 61, 20)}),
    ]),
    html.Div([
        html.Label("Adjust Coordinates of Point B:"),
        dcc.Slider(id="slider-bx", min=-60, max=60, step=0.1, value=19.71,
                   marks={i: str(i) for i in range(-60, 61, 20)}),
        dcc.Slider(id="slider-by", min=-60, max=60, step=0.1, value=3.79,
                   marks={i: str(i) for i in range(-60, 61, 20)}),
    ]),
    html.Div([
        html.Label("Adjust Coordinates of Point C:"),
        dcc.Slider(id="slider-cx", min=-60, max=60, step=0.1, value=40.96,
                   marks={i: str(i) for i in range(-60, 61, 20)}),
        dcc.Slider(id="slider-cy", min=-60, max=60, step=0.1, value=6.62,
                   marks={i: str(i) for i in range(-60, 61, 20)}),
    ]),
])

# Callback to update the 2D plot
@app.callback(
    Output("2d-plot", "figure"),
    [Input("slider-ax", "value"), Input("slider-ay", "value"),
     Input("slider-bx", "value"), Input("slider-by", "value"),
     Input("slider-cx", "value"), Input("slider-cy", "value")]
)
def update_plot(ax, ay, bx, by, cx, cy):
    # Points A, B, C
    A = np.array([ax, ay])
    B = np.array([bx, by])
    C = np.array([cx, cy])
    
    # Generate sequences
    sequence_A = generate_sequence(A, M)
    sequence_B = generate_sequence(B, M)
    sequence_C = generate_sequence(C, M)
    
    # Get eigenvalues and eigenvectors
    eigenvalues, eigenvectors = np.linalg.eig(M)

    # Create plot
    fig = go.Figure()

    # Add sequences
    fig.add_trace(go.Scatter(
        x=sequence_A[:, 0], y=sequence_A[:, 1],
        mode='lines+markers',
        name=f'Sequence A (A: [{A[0]:.2f}, {A[1]:.2f}])',
        marker=dict(size=5, color='red')
    ))
    fig.add_trace(go.Scatter(
        x=sequence_B[:, 0], y=sequence_B[:, 1],
        mode='lines+markers',
        name=f'Sequence B (B: [{B[0]:.2f}, {B[1]:.2f}])',
        marker=dict(size=5, color='blue')
    ))
    fig.add_trace(go.Scatter(
        x=sequence_C[:, 0], y=sequence_C[:, 1],
        mode='lines+markers',
        name=f'Sequence C (C: [{C[0]:.2f}, {C[1]:.2f}])',
        marker=dict(size=5, color='green')
    ))

    # Add infinite eigenvectors
    for i in range(2):
        x_values = np.linspace(-1000 * eigenvectors[0, i], 1000 * eigenvectors[0, i], 2)
        y_values = np.linspace(-1000 * eigenvectors[1, i], 1000 * eigenvectors[1, i], 2)
        fig.add_trace(go.Scatter(
            x=x_values,
            y=y_values,
            mode='lines',
            line=dict(color='purple', width=2, dash='dash'),
            name=f'Eigenvector {i+1}'
        ))

    # Add points
    fig.add_trace(go.Scatter(
        x=[A[0]], y=[A[1]],
        mode='markers+text',
        marker=dict(size=10, color='red'),
        text=['A'],
        name='Point A'
    ))
    fig.add_trace(go.Scatter(
        x=[B[0]], y=[B[1]],
        mode='markers+text',
        marker=dict(size=10, color='blue'),
        text=['B'],
        name='Point B'
    ))
    fig.add_trace(go.Scatter(
        x=[C[0]], y=[C[1]],
        mode='markers+text',
        marker=dict(size=10, color='green'),
        text=['C'],
        name='Point C'
    ))

    # Layout settings
    fig.update_layout(
        xaxis=dict(title='X', range=[-60, 60]),
        yaxis=dict(title='Y', range=[-60, 60]),
        title='Interactive 2D Transformation with Infinite Eigenvectors',
        showlegend=True
    )

    return fig

# Run the app
if __name__ == "__main__":
    app.run_server(debug=True)
